# Can we code a deep reasearch agent?(Basic Level)
# By - Aditya Bhatt

# TODO:

# 1.Ask the user what exactly they need a research on?(a simple follow up question)
# 2.Getting a lot of bugs on environment level will need to migate to proper py script


# Enter the Research Topic

In [1]:
query="On what topic you want to do deep research on ?"
# n=input(query)
n="Indian Food"
print("User has asked for research on",n)

User has asked for research on Indian Food


# Install the necessary libraries


In [7]:
pip uninstall -q openai pydantic 

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
! pip install google-search-results

# openai 1.42.0 and pydantic 2.8.2

In [8]:
pip install -q openai pydantic --upgrade


Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.


In [5]:
pip install python-dotenv


  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


# Set the client

In [11]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()
try:
    client = AzureOpenAI(
                        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
                        api_version = "2024-10-21",
                        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") or "default"
                        )
    
    print("Client created successfully")
except Exception as e:
    print(e)
    print("Error in creating the client")


Client.__init__() got an unexpected keyword argument 'proxies'
Error in creating the client


# Test the client

In [18]:
try:
    response = client.chat.completions.create(
        model="mini-deployment", # model = "deployment_name".
        messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Hi?"}
    ]
    )

    print(response.choices[0].message.content)
    print("Basic API call successful")
except Exception as e:
    print(e)
    print("Basic API call failed")


Hello! How can I assist you today?
Basic API call successful


# Convert the user query into a search query

# Concept used :Structured Output and Pydantic way of doing it


In [31]:
from pydantic import BaseModel
from openai import AzureOpenAI


class SearchQuery(BaseModel):
    """
    This is the search query that will be used to search the web
    """
    query: str

prompt1 = f"You need to convert the user query into a search query (which will be used to search the web) for the user query: {n}"

print(prompt1)
completion = client.beta.chat.completions.parse(
    model="mini-deployment",  # replace with the model deployment name of your gpt-4o 2024-08-06 deployment
    messages=[
        {"role": "system", "content": "You are a helpful assistant that converts user queries into search queries."},
        {"role": "user", "content": prompt1},
    ],
    response_format=SearchQuery,
)

search_query = completion.choices[0].message.parsed
query = search_query.query
print(search_query.query)

You need to convert the user query into a search query (which will be used to search the web) for the user query: Indian Food
Indian Food recipes, dishes, and cuisine


# Once we have the search query , we need a plan to break down the query into smaller pieces which will be used to define a report

In [34]:
class Plan(BaseModel):
    """
    This will breakdown the query into smaller pieces which will be used to define a report (word file, or pdf file (not sure yet))
    """
    items: list[str]  # A list of items separated by commas

prompt2 = f'''
As a research assistant, your task is to thoroughly break down the user query into detailed, manageable topics that will serve as the foundation for a comprehensive report. Ensure that each topic is relevant and contributes to a well-structured analysis.

Example: User query: Indian Food\n\nReturn the following topics: [Overview of Indian cuisine, Historical influences on Indian food, Regional variations in Indian cuisine, Cultural significance of Indian food, Contemporary trends in Indian dining, Health aspects of Indian food]

User query: {n}
'''
print(prompt2)

completion = client.beta.chat.completions.parse(
    model="mini-deployment",  
    messages=[
        {"role": "system", "content": "You are a research assistant that breaks down user queries into detailed, manageable topics."},
        {"role": "user", "content": prompt2},
    ],
    response_format=Plan,
)
list_of_topics = completion.choices[0].message.parsed.items



As a research assistant, your task is to thoroughly break down the user query into detailed, manageable topics that will serve as the foundation for a comprehensive report. Ensure that each topic is relevant and contributes to a well-structured analysis.

Example: User query: Indian Food

Return the following topics: [Overview of Indian cuisine, Historical influences on Indian food, Regional variations in Indian cuisine, Cultural significance of Indian food, Contemporary trends in Indian dining, Health aspects of Indian food]

User query: Indian Food



In [35]:
for i in list_of_topics:
    print("This is the topic:",i)

This is the topic: Overview of Indian cuisine
This is the topic: Historical influences on Indian food
This is the topic: Regional variations in Indian cuisine
This is the topic: Common ingredients used in Indian cooking
This is the topic: Popular Indian dishes and their significance
This is the topic: Cultural significance of Indian food
This is the topic: Religious and dietary restrictions related to Indian cuisine
This is the topic: Contemporary trends in Indian dining
This is the topic: Influence of globalization on Indian food
This is the topic: Health aspects of Indian food
This is the topic: Indian food in the diaspora
This is the topic: Cooking methods and techniques in Indian cuisine


# Once we have the list of topics, we can use these topics to search the web and get the relevant information.

In [36]:
list_of_topics

['Overview of Indian cuisine',
 'Historical influences on Indian food',
 'Regional variations in Indian cuisine',
 'Common ingredients used in Indian cooking',
 'Popular Indian dishes and their significance',
 'Cultural significance of Indian food',
 'Religious and dietary restrictions related to Indian cuisine',
 'Contemporary trends in Indian dining',
 'Influence of globalization on Indian food',
 'Health aspects of Indian food',
 'Indian food in the diaspora',
 'Cooking methods and techniques in Indian cuisine']

In [57]:
import ssl
import certifi

ssl_context = ssl.create_default_context(cafile=certifi.where())


In [13]:
import os
import ssl
import requests
from urllib3.exceptions import InsecureRequestWarning

# Suppress only the single InsecureRequestWarning
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

# Multiple SSL bypass methods
ssl._create_default_https_context = ssl._create_unverified_context

def get_google_urls(query, api_key, num_urls=5):
    base_url = "https://serpapi.com/search.json"
    
    params = {
        "engine": "google",
        "q": query,
        "google_domain": "google.com",
        "gl": "us",
        "hl": "en",
        "api_key": api_key,
        "num": 10,  # Request more results to ensure we get enough
        "start": 0  # Start from the first result
    }
    
    try:
        response = requests.get(base_url, params=params, verify=False)
        response.raise_for_status()
        results = response.json()
        
        urls = []
        if "organic_results" in results:
            urls = [result['link'] for result in results["organic_results"]]
            
            # If we still don't have enough results, make another request
            if len(urls) < num_urls:
                params["start"] = len(urls)  # Start from where we left off
                response = requests.get(base_url, params=params, verify=False)
                more_results = response.json()
                if "organic_results" in more_results:
                    urls.extend([result['link'] for result in more_results["organic_results"]])
            
            return urls[:num_urls]  # Return exactly num_urls links
            
    except Exception as e:
        print(f"Error: {e}")
        return []

# Your API key
api_key = "4040378d372e7d863af3811b19de34149fad33254c776cc2bff1b6071a8a87f9"

# Get the URLs
urls = get_google_urls(list_of_topics[0], api_key)

# Print the URLs
print(urls)

NameError: name 'list_of_topics' is not defined

In [12]:
for i in urls:
    print(i)

https://sloanreview.mit.edu/article/five-trends-in-ai-and-data-science-for-2025/
https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends
https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/
https://www.digitalocean.com/resources/articles/ai-trends
https://www.newark.com/latest-trends-in-artificial-intelligence
